# TIEGCM


The kamodofied tiegcm requires the pytiegcm reader available on github:

    pip install -e git+https://github.com/asher-pembroke/pytiegcm.git#egg=pytiegcm

The TIEGCM reader also requires `netcdf4` (and numpy, scipy, and pandas, but those are required by kamodo)

    pip install netcdf4

!!! warning
    This is a work in progress!

In [1]:
filename = '/Users/apembrok/Work/TIEGCM/sample_data/jasoon_shim_040118_IT_1/s001.nc' # ccmc run (no zgmid)
fname = '/Users/apembrok/Downloads/2013.03.01.tie-gcm.data/s003.nc' # (no zgmid)

In [2]:
from kamodo import readers

In [3]:
from kamodo.readers.tiegcm import TIEGCM_Kamodo

By default, the kamodofied TIEGCM reader will load all 3d and 4d variables from the model. For performance reasons, you may want to only include certain variables for analysis. To do so, pass a list of variables in addition to the filename:

In [4]:
tiegcm = TIEGCM_Kamodo(fname, ['TN', 'UN', 'VN', 'EFLUX', 'Z'])


initializing tiegcm with /Users/apembrok/Downloads/2013.03.01.tie-gcm.data/s003.nc


/Users/apembrok/git/kamodo/docs/notebooks/src/pytiegcm/tiegcm/tiegcm.py:213: UserWarning:

cannot be safely cast to variable data type



registered 5 variables


In [5]:
tiegcm

TIEGCM_Kamodo([(TN(t, ilev, lat, lon),
                <function wrapped(t=array([ 980., 1000., 1020., 1040., 1060., 1080., 1100., 1120., 1140.,
       1160., 1180., 1200., 1220., 1240., 1260., 1280., 1300., 1320.,
       1340., 1360., 1380., 1400., 1420., 1440.]), ilev=array([-7. , -6.5, -6. , -5.5, -5. , -4.5, -4. , -3.5, -3. , -2.5, -2. ,
       -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,
        4. ,  4.5,  5. ,  5.5,  6. ,  6.5,  7. ]), lat=array([-90. , -87.5, -82.5, -77.5, -72.5, -67.5, -62.5, -57.5, -52.5,
       -47.5, -42.5, -37.5, -32.5, -27.5, -22.5, -17.5, -12.5,  -7.5,
        -2.5,   2.5,   7.5,  12.5,  17.5,  22.5,  27.5,  32.5,  37.5,
        42.5,  47.5,  52.5,  57.5,  62.5,  67.5,  72.5,  77.5,  82.5,
        87.5,  90. ]), lon=array([-180., -175., -170., -165., -160., -155., -150., -145., -140.,
       -135., -130., -125., -120., -115., -110., -105., -100.,  -95.,
        -90.,  -85.,  -80.,  -75.,  -70.,  -65.,  -60.,  -55.,  -50.,
        -45

Below is an attempt to use function composition to solve variables as a function of height.

In [6]:
from scipy.interpolate import interp1d

In [7]:
import numpy as np
z = np.squeeze(tiegcm.Z(tiegcm._time[0], lat = 0, lon = 0))

In [26]:
tiegcm['T_N(t, z, lat, lon)'] = 'TN(t,ilev, lat, lon)'

In [27]:
tiegcm

TIEGCM_Kamodo([(TN(t, ilev, lat, lon),
                <function wrapped(t=array([ 980., 1000., 1020., 1040., 1060., 1080., 1100., 1120., 1140.,
       1160., 1180., 1200., 1220., 1240., 1260., 1280., 1300., 1320.,
       1340., 1360., 1380., 1400., 1420., 1440.]), ilev=array([-7. , -6.5, -6. , -5.5, -5. , -4.5, -4. , -3.5, -3. , -2.5, -2. ,
       -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,
        4. ,  4.5,  5. ,  5.5,  6. ,  6.5,  7. ]), lat=array([-90. , -87.5, -82.5, -77.5, -72.5, -67.5, -62.5, -57.5, -52.5,
       -47.5, -42.5, -37.5, -32.5, -27.5, -22.5, -17.5, -12.5,  -7.5,
        -2.5,   2.5,   7.5,  12.5,  17.5,  22.5,  27.5,  32.5,  37.5,
        42.5,  47.5,  52.5,  57.5,  62.5,  67.5,  72.5,  77.5,  82.5,
        87.5,  90. ]), lon=array([-180., -175., -170., -165., -160., -155., -150., -145., -140.,
       -135., -130., -125., -120., -115., -110., -105., -100.,  -95.,
        -90.,  -85.,  -80.,  -75.,  -70.,  -65.,  -60.,  -55.,  -50.,
        -45

In [36]:
import numpy as np

@np.vectorize
def lev(self, t, z, lat, lon):
    # 1) meshgrid and squeeze the shapes of time, lat, lon if they are not the same
    # 2) calculate z_levels for all points
    # 3) construct rgi for (time, z_level, lat, lon)
    # 4) interpolate over z
    
    z_levels = self.Z(time = t, lat = lat, lon = lon)
    level = scipy.interpolate.interp1d(z_levels, ilev, bounds_error=False)
    return level(z)
tiegcm.lev = lev

In [28]:
tiegcm['ilev'] = lev

In [32]:
tiegcm['T_N(t, z, lat, lon)'] = 'TN(t, ilev, lat, lon)'

In [33]:
tiegcm

TIEGCM_Kamodo([(TN(t, ilev, lat, lon),
                <function wrapped(t=array([ 980., 1000., 1020., 1040., 1060., 1080., 1100., 1120., 1140.,
       1160., 1180., 1200., 1220., 1240., 1260., 1280., 1300., 1320.,
       1340., 1360., 1380., 1400., 1420., 1440.]), ilev=array([-7. , -6.5, -6. , -5.5, -5. , -4.5, -4. , -3.5, -3. , -2.5, -2. ,
       -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,
        4. ,  4.5,  5. ,  5.5,  6. ,  6.5,  7. ]), lat=array([-90. , -87.5, -82.5, -77.5, -72.5, -67.5, -62.5, -57.5, -52.5,
       -47.5, -42.5, -37.5, -32.5, -27.5, -22.5, -17.5, -12.5,  -7.5,
        -2.5,   2.5,   7.5,  12.5,  17.5,  22.5,  27.5,  32.5,  37.5,
        42.5,  47.5,  52.5,  57.5,  62.5,  67.5,  72.5,  77.5,  82.5,
        87.5,  90. ]), lon=array([-180., -175., -170., -165., -160., -155., -150., -145., -140.,
       -135., -130., -125., -120., -115., -110., -105., -100.,  -95.,
        -90.,  -85.,  -80.,  -75.,  -70.,  -65.,  -60.,  -55.,  -50.,
        -45

In [17]:
tiegcm['eflux'] = 'EFLUX(t, lat, lon)'

In [18]:
tiegcm

TIEGCM_Kamodo([(TN(t, ilev, lat, lon),
                <function wrapped(t=array([ 980., 1000., 1020., 1040., 1060., 1080., 1100., 1120., 1140.,
       1160., 1180., 1200., 1220., 1240., 1260., 1280., 1300., 1320.,
       1340., 1360., 1380., 1400., 1420., 1440.]), ilev=array([-7. , -6.5, -6. , -5.5, -5. , -4.5, -4. , -3.5, -3. , -2.5, -2. ,
       -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,
        4. ,  4.5,  5. ,  5.5,  6. ,  6.5,  7. ]), lat=array([-90. , -87.5, -82.5, -77.5, -72.5, -67.5, -62.5, -57.5, -52.5,
       -47.5, -42.5, -37.5, -32.5, -27.5, -22.5, -17.5, -12.5,  -7.5,
        -2.5,   2.5,   7.5,  12.5,  17.5,  22.5,  27.5,  32.5,  37.5,
        42.5,  47.5,  52.5,  57.5,  62.5,  67.5,  72.5,  77.5,  82.5,
        87.5,  90. ]), lon=array([-180., -175., -170., -165., -160., -155., -150., -145., -140.,
       -135., -130., -125., -120., -115., -110., -105., -100.,  -95.,
        -90.,  -85.,  -80.,  -75.,  -70.,  -65.,  -60.,  -55.,  -50.,
        -45

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
